### Algorithms and Applications in Social Networks - HW2

Packages imports for all questions' code below:

In [19]:
import networkx as nx

### Q1
a. Implement Newman-Girvan algorithm for non-overlapping communities. The algorithm should receive a network and parameter k (number of communities) and return the communities.

In [20]:
def newman_girvan(G, k):
    g = G.copy() #in order not to affect the original graph.
    communities_num = nx.number_connected_components(g)
    while communities_num < k:
        ebc = nx.edge_betweenness_centrality(g) #compute edge betweenness for all edges.
        max_bw = max(ebc.values()) #find max value of edge betweenness.
        #find arbitrary edge with the largest edge betweenes and remove it.
        max_val = 0.0
        for edge, val in ebc.items():
            if val > max_val:
                max_val = val
                max_edge = edge
        g.remove_edge(*max_edge) #remove the chosen edge.
        communities_num = nx.number_connected_components(g) #recalculate the nubmer of cc. 
    communities = nx.connected_components(g)
    return communities

b. Run this algorithm on the biggest connected component of the following dataset: https://bit.ly/2KLHN60 (with k=3). Each line of the file represents an edge between two nodes.

In [21]:
def run_communities_algorithm(file, algorithm, k):
    g = nx.read_edgelist(file)
    biggest_cc_set = max(nx.connected_components(g), key=len)
    biggest_cc_graph = g.subgraph(biggest_cc_set).copy()
    communities = [c for c in algorithm(biggest_cc_graph, k)]
    algorithm_name = str(algorithm).split()[1].title().replace("_", " ")
    print(f"The communities found by {algorithm_name} are:")
    for i, community in enumerate(communities):
        print(f"community #{i} :")
        print(set(community))

In [22]:
run_communities_algorithm('communities.txt', newman_girvan, 3)

The communities found by Newman Girvan are:
community #0 :
{'79', '30', '257', '69', '119', '77', '268', '207', '236', '272', '118', '283', '208', '265', '269', '161', '332', '117', '286', '36', '94', '184', '64', '38', '203', '53', '120', '142', '276', '85', '40', '315', '130', '303', '106', '213', '83', '26', '291', '135', '153', '58', '217', '80', '223', '148', '72', '98', '248', '204', '202', '339', '322', '48', '185', '221', '281', '251', '108', '136', '54', '81', '126', '123', '274', '163', '141', '57', '165', '196', '190', '334', '176', '313', '330', '308', '229', '254', '125', '31', '88', '288', '249', '200', '338', '316', '280', '173', '150', '329', '10', '172', '63', '242', '25', '183', '270', '55', '158', '246', '317', '340', '50', '300', '222', '105', '1', '238', '39', '206', '128', '186', '75', '146', '60', '271', '285', '344', '113', '56', '51', '346', '239', '103', '159', '235', '139', '224', '260', '13', '258', '24', '171', '166', '133', '96', '231', '134', '164', '297'

### Q2
a. Implement k-clique communities detection algorithm. The algorithm should receive a network and parameter k and return the communities.

In [23]:
def k_clique_communities(g, k):
    '''
    A k-clique community ditection implementation using percolation method (Palla et al. 2005).
    1. Find all maximal cliques
    2. Create clique overlap matrix
    3. Threshold matrix with k-1
    4. Communities are connected components
    '''
    maximal_cliques = [frozenset(c) for c in nx.find_cliques(g)] #step (1) - find all maximal cliques.
    mat = create_threshold_clique_overlap_matrix(maximal_cliques, k) #steps (2), (3).
    communities = find_communities_as_cc(maximal_cliques, mat) #step (4)
    return communities

def create_threshold_clique_overlap_matrix(maximal_cliques, k):
    #instead of creating the overlap matrix as an intermediate step,
    #we create the final threshold matrix.
    mat = [] #the threshold clique overlap matrix.
    for clique_1 in maximal_cliques:
        mat.append([]) #add column to the threshold clique overlap matrix.
        if len(clique_1) < k: #the threshold is k-1 and cliques with size under k can't pass it.
            for i in range(len(maximal_cliques)):
                mat[-1].append(0)
            #this way there is no need to check the diagonal values with a threshold k,
            #while in other cells we use threshold of k-1.
        else: #len(clique1) >= k
            for clique_2 in maximal_cliques:
                val = 0
                intersection = len(clique_1.intersection(clique_2))
                if intersection >= k-1: #the threshold is k-1.
                    val = 1
                mat[-1].append(val)
    return mat

def find_communities_as_cc(maximal_cliques, mat):
    cc_g = nx.Graph()
    #a new graph for connected components derived from the threshold clique overlap matrix.
    for row, clique_1 in enumerate(maximal_cliques):
        for col, clique_2 in enumerate(maximal_cliques):
            if int(mat[row][col]) == 1:
                cc_g.add_edge(clique_1, clique_2)
    communities = [frozenset.union(*connected_component) for connected_component in nx.connected_components(cc_g)]
    return communities


b. Run this algorithm on the biggest connected component of the following dataset: https://bit.ly/2KLHN60 (with k=4).

In [24]:
run_communities_algorithm('communities.txt', k_clique_communities, 4)

The communities found by K Clique Communities are:
community #0 :
{'79', '30', '257', '69', '119', '77', '268', '272', '236', '118', '208', '265', '161', '332', '36', '94', '184', '64', '38', '203', '53', '142', '276', '85', '40', '315', '130', '303', '106', '213', '26', '291', '135', '217', '80', '223', '148', '72', '98', '248', '204', '339', '322', '48', '185', '221', '281', '251', '108', '136', '54', '126', '123', '274', '163', '141', '57', '165', '196', '334', '176', '313', '330', '308', '229', '254', '31', '88', '249', '200', '338', '280', '150', '329', '10', '172', '242', '63', '25', '55', '158', '246', '317', '50', '340', '300', '222', '105', '238', '1', '39', '128', '186', '146', '75', '60', '271', '285', '344', '113', '56', '346', '239', '103', '159', '139', '224', '13', '258', '24', '171', '133', '134', '231', '96', '297', '320', '3', '9', '295', '29', '82', '302', '298', '59', '187', '178', '65', '250', '261', '101', '121', '66', '67', '197', '314', '347', '27', '129', '127'